In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta


from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import LightSource
from matplotlib import cm

from functions.plot_prod import plot_prod

from bentso import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Small test now

In [159]:
rolling=1 # rolling period (choose a multiple of 7 if days for best results)
country='de'

prod_df_2021, load_df, colors_2021 = plot_prod(country=country, # country code
                             year='2021',  # year
                             display=False, # display chart (or just save data)
                             load=True,    # show load
                             cumul=False,  # cumulative
                             rolling=rolling,   # rolling average, in timesteps
                             lang='en',    # language
                             step='hour')

https://energy-charts.info/charts/power/data/de/year_2021.json


In [160]:
cl = CachingDataClient()
inst_cap_2021 = cl.get_capacity(country.upper(),2021)

Using data directory C:\Users\Gibon\AppData\Local\bonsai\bentso


In [169]:
inst_cap_2021.columns = [c.capitalize() for c in inst_cap_2021.columns]
prod_df_2021.columns  = [c.capitalize() for c in prod_df_2021.columns]

In [180]:
inst_cap_2021.rename({'Hydro run-of-river and poundage':'Hydro run-of-river',
                      'Fossil brown coal/lignite':'Fossil brown coal'}, axis=1, inplace=True)
try:
    inst_cap_2021['Wind'] = inst_cap_2021[['Wind offshore','Wind onshore']].sum(1)
except:
    pass

In [181]:
colors_2021_df = pd.DataFrame(colors_2021)
colors_2021_df.rename({'Hydro run-of-river and poundage':'Hydro run-of-river',
                      'Hydro Run-of-River':'Hydro run-of-river'}, axis=1, inplace=True)
colors_2021_df[[c.capitalize() for c in colors_2021_df.columns]] = colors_2021_df

In [185]:
intersect = inst_cap_2021.columns & prod_df_2021.columns

In [186]:
inst_cap_2021 = inst_cap_2021[intersect]
prod_df_2021 = prod_df_2021[intersect]

In [187]:
if country=='de':
    prod_df_2021 = prod_df_2021.groupby(pd.Grouper(freq='H')).sum()/4

In [188]:
cumul_2021 = prod_df_2021.cumsum(axis=0)

In [189]:
order = cumul_2021.iloc[-1,:].sort_values(ascending = False).index

In [190]:
_xx = inst_cap_2021[order].cumsum(axis=1)
_x = [0,0] + pd.concat([_xx]*2,axis=0).unstack().values.tolist()
_y = range(0,cumul_2021.shape[0])
X, Y = np.meshgrid(_x, _y)
Z = pd.concat([cumul_2021]*2, keys=['start','stop']).unstack(0)

In [191]:
cap = pd.concat([inst_cap_2021.iloc[[0]]] *  Z.shape[0]).set_index(Z.index)

In [192]:
Z = Z.div(cap,level=0)*1000

Z[('last','stop')] = 0
Z[('first','start')] = 0
Z = Z[['first'] + order.tolist() + ['last']]

# Z = Z.loc[:,~(Z==np.inf).all()]

In [193]:
months = prod_df_2021.groupby(pd.Grouper(freq='M')).sum().index.strftime('%B')[1:]
ticks=[]

for i,d in enumerate(prod_df_2021.index.day):
    if i in range(1,prod_df_2021.shape[0]-1):
        if prod_df_2021.index[i].day - prod_df_2021.index[i+1].day > 2:
            ticks.append(i)
ticks.append(i)

In [194]:
%matplotlib qt

fig = plt.figure(figsize=(30, 30))
ax1 = fig.add_subplot(111, projection='3d')

for i,t in enumerate(order):
    
    s = slice(2*i, 2*(i+2), None)
    
    # Sides and top
    ax1.plot_surface(X[:,s],
                     Y[:,s],
                     Z[['first',t,'last']],
                    color=colors_2021_df[t])
    
    ax1.text3D(X[0,2*i] + (X[0,2*i+2] - X[0,2*i])/2,
               _y[-1], Z[t].iloc[-1,-1] + 100, t, zdir='z')
    
    # Last slice
    verts = [list(zip(X[-1,s],
                      Y[-1,s],
                      Z[['first',t,'last']].iloc[-1,:]))]
    ax1.add_collection3d(Poly3DCollection(verts,facecolor=colors_2021_df[t]))
    
    # Bottom
    verts = [list(zip(X[-1,s],
                      [1,*Y[-1,-2:],1],
                      [0,0,0,0]))]
    ax1.add_collection3d(Poly3DCollection(verts,facecolor=colors_2021_df[t]))
    
ax1.set_xlabel('Installed capacity (GW)')
ax1.set_yticks(ticks)
ax1.set_yticklabels(months)
ax1.set_ylabel('Time')

ax1.set_zlabel('Load hours')

Text(0.5, 0, 'Load hours')